"""Full Pipeline Demo (Stages 0-2 with LLM-enabled service).

Flow:
1. Stage 0: Project Setup (ProjectContext)
2. Approve context
3. Stage 1: Problem Framing (ProblemFraming + ConceptModel)
4. Approve framing & concept model
5. Stage 2: Research Questions (ResearchQuestionSet)
6. Display summaries of each artifact

Assumes .env configured with LLM provider (mock or openai via OpenRouter).
"""


In [16]:
import os
import logging
from dotenv import load_dotenv, find_dotenv

# Force load .env (prints path for verification)
dotenv_path = find_dotenv(usecwd=True)
if not dotenv_path:
    raise FileNotFoundError("`.env` not found. Place it in project root.")
load_dotenv(dotenv_path=dotenv_path)

logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Continue with imports AFTER env is loaded
from src.controller import PipelineController
from src.services import FilePersistenceService, IntelligentModelService
from src.models import (
    ProjectContext, ProblemFraming, ConceptModel,
    ResearchQuestionSet, SearchConceptBlocks, ApprovalStatus
)

RAW_IDEA = (
    """# Search Queries for Scoping Review

**Research Topic:** Domain Shift and Generalization in Deep Learning-Based Crop Disease and Pest Detection

**Research Question:** What is the landscape of research on domain shift and generalization  in deep learning-based crop disease and pest detection? What types of  evidence, methodologies, and gaps exist?"""
)

def approve(controller, project_id, artifact_type, artifact_class, edits=None):
    controller.approve_artifact(
        project_id=project_id,
        artifact_type=artifact_type,
        artifact_class=artifact_class,
        edits=edits or {},
        approval_status=ApprovalStatus.APPROVED
    )

In [17]:
print("\n=== FULL PIPELINE DEMO (Stages 0-3) ===\n")
model_service = IntelligentModelService()
persistence = FilePersistenceService(base_dir="./data")
controller = PipelineController(model_service=model_service, persistence_service=persistence)

# Stage 0
print("[Stage 0] Generating ProjectContext...")
stage0 = controller.start_project(raw_idea=RAW_IDEA)
ctx: ProjectContext = stage0.draft_artifact
print(f"Project ID: {ctx.id}")
print(f"Title: {ctx.title}")
print(f"Keywords: {ctx.initial_keywords}\n")
approve(controller, ctx.id, "ProjectContext", ProjectContext)


2025-11-21 10:38:57,939 - src.services.llm_provider - INFO - Using custom base URL: https://openrouter.ai/api/v1



=== FULL PIPELINE DEMO (Stages 0-3) ===



2025-11-21 10:38:58,408 - src.services.intelligent_model_service - INFO - IntelligentModelService initialized with provider: LLMProvider.OPENAI
2025-11-21 10:38:58,410 - src.services.intelligent_model_service - INFO - Generating project context from raw idea...
2025-11-21 10:38:58,412 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'timeout': 30, 'files': None, 'idempotency_key': 'stainless-python-retry-80fd244a-3788-495e-9cd1-797c7f8169db', 'json_data': {'messages': [{'role': 'system', 'content': 'You are an expert Research Methodologist with 20+ years of experience in systematic reviews and meta-analyses.\n\nYour role is to help researchers:\n- Frame research problems with academic rigor\n- Define clear, measurable research objectives\n- Establish appropriate scope boundaries\n- Identify key concepts and relationships\n\nYou ALWAYS output strictly valid JSON with no additional text.\nYou are precise, avoiding vague terms like "AI" when 

[Stage 0] Generating ProjectContext...


2025-11-21 10:38:59,768 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 21 Nov 2025 18:39:00 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Content-Encoding', b'gzip'), (b'Access-Control-Allow-Origin', b'*'), (b'Vary', b'Accept-Encoding'), (b'Permissions-Policy', b'payment=(self "https://checkout.stripe.com" "https://connect-js.stripe.com" "https://js.stripe.com" "https://*.js.stripe.com" "https://hooks.stripe.com")'), (b'Referrer-Policy', b'no-referrer, strict-origin-when-cross-origin'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9a224ea5fb743369-AAE')])
2025-11-21 10:38:59,770 - httpx - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-21 10:38:59,770 - httpcore.http11 - DEBUG - receive_response_body.started request=<Request [b'POST']>
2025-11-21 10:39:03,43

Project ID: project_9161c011
Title: A Scoping Review on Domain Shift and Generalization in Deep Learning for Crop Disease and Pest Detection
Keywords: ['domain shift', 'generalization', 'deep learning', 'crop disease detection', 'pest detection']



In [18]:
# Stage 1: Problem Framing + Concept Model
print("[Stage 1] Generating ProblemFraming & ConceptModel...")
stage1 = controller.run_stage("problem-framing", project_id=ctx.id)
framing: ProblemFraming = stage1.draft_artifact
concept_model: ConceptModel = stage1.extra_data.get("concept_model")
if framing is None:
    raise RuntimeError(f"Stage 1 failed: {stage1.validation_errors}")
print("Problem Statement:\n", framing.problem_statement)
print("Research Gap:\n", framing.research_gap)
print("Goals:", framing.goals)
print("Concepts:", [c.label for c in concept_model.concepts])
approve(controller, ctx.id, "ProblemFraming", ProblemFraming)
approve(controller, ctx.id, "ConceptModel", ConceptModel)


2025-11-21 10:39:19,396 - src.services.intelligent_model_service - INFO - Generating problem framing for: A Scoping Review on Domain Shift and Generalization in Deep Learning for Crop Disease and Pest Detection
2025-11-21 10:39:19,397 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'timeout': 30, 'files': None, 'idempotency_key': 'stainless-python-retry-479f067b-8f5b-4929-8ea9-50a5d1adc7bc', 'json_data': {'messages': [{'role': 'system', 'content': 'You are a strict Senior Research Supervisor with expertise in research methodology.\n\nYour job is to find flaws in research framing:\n- Vague or undefined terms (e.g., "AI" instead of specific model types)\n- Overly broad scope that makes research infeasible\n- Missing operational definitions\n- Unclear or unmeasurable outcomes\n- Scope creep or conflicting objectives\n\nYou are NEVER satisfied with generic terms. You demand specificity.\nYou rate feasibility honestly - a score of 10 means "re

[Stage 1] Generating ProblemFraming & ConceptModel...


2025-11-21 10:39:22,619 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 21 Nov 2025 18:39:23 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Content-Encoding', b'gzip'), (b'Access-Control-Allow-Origin', b'*'), (b'Vary', b'Accept-Encoding'), (b'Permissions-Policy', b'payment=(self "https://checkout.stripe.com" "https://connect-js.stripe.com" "https://js.stripe.com" "https://*.js.stripe.com" "https://hooks.stripe.com")'), (b'Referrer-Policy', b'no-referrer, strict-origin-when-cross-origin'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9a224f293d0a3351-AAE')])
2025-11-21 10:39:22,620 - httpx - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-21 10:39:22,621 - httpcore.http11 - DEBUG - receive_response_body.started request=<Request [b'POST']>
2025-11-21 10:39:29,14

Problem Statement:
 This research seeks to clarify the impact of data distribution shifts and environmental variability on the performance of convolutional neural networks (CNNs) for crop disease and pest detection. It aims to identify specific factors influencing model generalization, including robustness and transferability across diverse agricultural datasets.
Research Gap:
 Current literature lacks a comprehensive analysis of how domain shifts impact the generalization capabilities of specific deep learning methodologies in agricultural contexts, particularly regarding the effectiveness of CNNs and transfer learning techniques in varying environmental conditions.
Goals: ['Evaluate the influence of data distribution shifts and environmental variability on the robustness of CNNs in crop disease detection by analyzing performance metrics such as accuracy and F1 score within a 6-month timeframe.', 'Compare the transferability of CNN models trained on one dataset to their performance on

In [19]:
# Stage 2: Research Questions
print("[Stage 2] Generating ResearchQuestionSet...")
stage2 = controller.run_stage("research-questions", project_id=ctx.id)
rq_set: ResearchQuestionSet = stage2.draft_artifact
if rq_set is None:
    raise RuntimeError(f"Stage 2 failed: {stage2.validation_errors}")
print(f"Generated {len(rq_set.questions)} research questions:")
for q in rq_set.questions:
    print(f" - ({q.priority}) [{q.type}] {q.text}")
approve(controller, ctx.id, "ResearchQuestionSet", ResearchQuestionSet)


2025-11-21 10:40:29,582 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'timeout': 30, 'files': None, 'idempotency_key': 'stainless-python-retry-ae6176a8-5d60-4544-b270-c3d2f7f14e42', 'json_data': {'messages': [{'role': 'system', 'content': 'You are an expert Research Methodologist with 20+ years of experience in systematic reviews and meta-analyses.\n\nYour role is to help researchers:\n- Frame research problems with academic rigor\n- Define clear, measurable research objectives\n- Establish appropriate scope boundaries\n- Identify key concepts and relationships\n\nYou ALWAYS output strictly valid JSON with no additional text.\nYou are precise, avoiding vague terms like "AI" when "Large Language Models" is meant.\nYou follow PICO framework (Population, Intervention, Comparison, Outcome) for clinical research.\n'}, {'role': 'user', 'content': 'Based on the approved problem framing, generate structured research questions.\n\nProblem Statem

[Stage 2] Generating ResearchQuestionSet...


2025-11-21 10:40:30,721 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 21 Nov 2025 18:40:31 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Content-Encoding', b'gzip'), (b'Access-Control-Allow-Origin', b'*'), (b'Vary', b'Accept-Encoding'), (b'Permissions-Policy', b'payment=(self "https://checkout.stripe.com" "https://connect-js.stripe.com" "https://js.stripe.com" "https://*.js.stripe.com" "https://hooks.stripe.com")'), (b'Referrer-Policy', b'no-referrer, strict-origin-when-cross-origin'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9a2250dfef9db6be-AAE')])
2025-11-21 10:40:30,722 - httpx - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-21 10:40:30,723 - httpcore.http11 - DEBUG - receive_response_body.started request=<Request [b'POST']>
2025-11-21 10:40:35,02

Generated 5 research questions:
 - (must_have) [explanatory] How do data distribution shifts affect the robustness of convolutional neural networks in detecting crop diseases and pests?
 - (must_have) [evaluative] What is the impact of environmental variability on the performance metrics (accuracy and F1 score) of CNNs for crop disease detection?
 - (must_have) [evaluative] To what extent do CNN models trained on one agricultural dataset transfer to unseen datasets with different environmental conditions?
 - (nice_to_have) [descriptive] What research gaps exist in the application of deep learning methodologies for agricultural pest detection based on a systematic review of relevant studies?
 - (nice_to_have) [explanatory] What factors influence the generalization of CNNs across diverse agricultural datasets in the context of crop disease and pest detection?


In [20]:
# Stage 3: Search Concept Expansion
print("[Stage 3] Generating SearchConceptBlocks...")
stage3 = controller.run_stage("search-concept-expansion", project_id=ctx.id)
blocks: SearchConceptBlocks = stage3.draft_artifact
if blocks is None:
    raise RuntimeError(f"Stage 3 failed: {stage3.validation_errors}")
print(f"Generated {len(blocks.blocks)} search concept blocks:")
for block in blocks.blocks:
    print(f" - {block.label}: {len(block.terms_included)} terms included")
    print(f"   Terms: {', '.join(block.terms_included[:6])}")
    if block.terms_excluded:
        print(f"   Excluded: {', '.join(block.terms_excluded)}")
approve(controller, ctx.id, "SearchConceptBlocks", SearchConceptBlocks)


2025-11-21 10:41:54,100 - src.services.intelligent_model_service - DEBUG - Stage3 search expansion prompt:
Based on the approved concepts and research questions, generate search concept blocks.

Concepts:
- Domain Shift (Methodology): Variability in data distribution due to changes in environmental conditions affecting crop disease and pest detection.
- Generalization (Outcome): The ability of deep learning models, particularly CNNs, to maintain performance across diverse datasets and unseen scenarios.
- Convolutional Neural Networks (Intervention): A class of deep learning algorithms applied specifically for image recognition tasks, such as detecting crop diseases and pests.
- Transfer Learning (Intervention): A method in deep learning where a model developed for a specific task is reused as the starting point for a model on a second task.
- Crop Disease and Pest Detection (Population): The application area focusing on identifying diseases and pests in crops using deep learning method

[Stage 3] Generating SearchConceptBlocks...


2025-11-21 10:41:55,076 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 21 Nov 2025 18:41:56 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Content-Encoding', b'gzip'), (b'Access-Control-Allow-Origin', b'*'), (b'Vary', b'Accept-Encoding'), (b'Permissions-Policy', b'payment=(self "https://checkout.stripe.com" "https://connect-js.stripe.com" "https://js.stripe.com" "https://*.js.stripe.com" "https://hooks.stripe.com")'), (b'Referrer-Policy', b'no-referrer, strict-origin-when-cross-origin'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9a2252f058c0b6c1-AAE')])
2025-11-21 10:41:55,077 - httpx - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-21 10:41:55,078 - httpcore.http11 - DEBUG - receive_response_body.started request=<Request [b'POST']>
2025-11-21 10:42:04,64

Generated 5 search concept blocks:
 - Domain Shift (Methodology): 4 terms included
   Terms: domain shift, data distribution shift, environmental variability, data drift
   Excluded: static data, fixed distribution
 - Generalization (Outcome): 4 terms included
   Terms: generalization, model robustness, performance consistency, transferable learning
   Excluded: overfitting, specificity
 - Convolutional Neural Networks (Intervention): 4 terms included
   Terms: CNN, convolutional networks, deep learning, image recognition algorithms
   Excluded: recurrent neural networks, traditional machine learning
 - Transfer Learning (Intervention): 4 terms included
   Terms: transfer learning, knowledge transfer, fine-tuning, pre-trained models
   Excluded: training from scratch, non-transferable models
 - Crop Disease and Pest Detection (Population): 4 terms included
   Terms: crop disease detection, pest identification, agricultural disease, crop health monitoring
   Excluded: animal disease, no

In [21]:
print(f"""### Final Summary
| Artifact | Status | Count |
|----------|--------|-------|
| ProjectContext | Approved | - |
| ProblemFraming | Approved | Goals: {len(framing.goals)} Concepts: {len(concept_model.concepts)} |
| ResearchQuestionSet | Approved | Questions: {len(rq_set.questions)} |
| SearchConceptBlocks | Approved | Blocks: {len(blocks.blocks)} |

Next Stage: Database Query Plan (Stage 4)
""")

### Final Summary
| Artifact | Status | Count |
|----------|--------|-------|
| ProjectContext | Approved | - |
| ProblemFraming | Approved | Goals: 3 Concepts: 5 |
| ResearchQuestionSet | Approved | Questions: 5 |
| SearchConceptBlocks | Approved | Blocks: 5 |

Next Stage: Database Query Plan (Stage 4)

